## Use ADQL and jointables to cross-match the Gaia sources in Koljonen et al. to ATNF and FAST

In [23]:
#imports
from astroquery.gaia import Gaia
from astropy.table import Table, QTable
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import Angle 
from astropy.coordinates import SkyCoord
import numpy as np

import eliminating

In [24]:
# load in tables Gaia has
tables = Gaia.load_tables(only_names=True)

INFO: Retrieving tables... [astroquery.utils.tap.core]
INFO: Parsing tables... [astroquery.utils.tap.core]
INFO: Done. [astroquery.utils.tap.core]


In [25]:
# see what tables Gaia has
for table in tables:
    print(table.name)

external.apassdr9
external.catwise2020
external.gaiadr2_astrophysical_parameters
external.gaiadr2_geometric_distance
external.gaiaedr3_distance
external.gaiaedr3_gcns_main_1
external.gaiaedr3_gcns_rejected_1
external.gaiaedr3_spurious
external.gaia_eso_survey
external.galex_ais
external.ravedr5_com
external.ravedr5_dr5
external.ravedr5_gra
external.ravedr5_on
external.ravedr6
external.sdssdr13_photoprimary
external.skymapperdr1_master
external.skymapperdr2_master
external.tmass_xsc
gaiadr1.aux_qso_icrf2_match
gaiadr1.ext_phot_zero_point
gaiadr1.allwise_best_neighbour
gaiadr1.allwise_neighbourhood
gaiadr1.gsc23_best_neighbour
gaiadr1.gsc23_neighbourhood
gaiadr1.ppmxl_best_neighbour
gaiadr1.ppmxl_neighbourhood
gaiadr1.sdss_dr9_best_neighbour
gaiadr1.sdss_dr9_neighbourhood
gaiadr1.tmass_best_neighbour
gaiadr1.tmass_neighbourhood
gaiadr1.ucac4_best_neighbour
gaiadr1.ucac4_neighbourhood
gaiadr1.urat1_best_neighbour
gaiadr1.urat1_neighbourhood
gaiadr1.cepheid
gaiadr1.phot_variable_time_serie

In [26]:
# get the metadata for gaiadr3.gaia_source
table_metadata = Gaia.load_table('gaiadr3.gaia_source')
print(table_metadata)

Retrieving table 'gaiadr3.gaia_source'
TAP Table name: gaiadr3.gaiadr3.gaia_source
Description: This table has an entry for every Gaia observed source as published with this data release. It contains the basic source parameters, in their final state as processed by the Gaia Data Processing and Analysis Consortium from the raw data coming from the spacecraft. The table is complemented with others containing information specific to certain kinds of objects (e.g.~Solar--system objects, non--single stars, variables etc.) and value--added processing (e.g.~astrophysical parameters etc.). Further array data types (spectra, epoch measurements) are presented separately via Datalink resources.
Num. columns: 152


In [27]:
# see what columns this table has
for column in table_metadata.columns:
    print(column.name)

solution_id
designation
source_id
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
parallax_over_error
pm
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pmra_pmdec_corr
astrometric_n_obs_al
astrometric_n_obs_ac
astrometric_n_good_obs_al
astrometric_n_bad_obs_al
astrometric_gof_al
astrometric_chi2_al
astrometric_excess_noise
astrometric_excess_noise_sig
astrometric_params_solved
astrometric_primary_flag
nu_eff_used_in_astrometry
pseudocolour
pseudocolour_error
ra_pseudocolour_corr
dec_pseudocolour_corr
parallax_pseudocolour_corr
pmra_pseudocolour_corr
pmdec_pseudocolour_corr
astrometric_matched_transits
visibility_periods_used
astrometric_sigma5d_max
matched_transits
new_matched_transits
matched_transits_removed
ipd_gof_harmonic_amplitude
ipd_gof_harmonic_phase
ipd_frac_multi_peak
ipd_frac_odd_win
ruwe
scan_direction_strength_k1
scan_di

In [28]:
# load in Koljonen source ids as a list 
koljonen_sourceids = [455282205716288384, 3831382647922429952, 5367876720979404288, 3990037124929068032, 6128369984328414336, 6140785016794586752,
            6098156298150016768, 4358428942492430336, 6025344817107454464, 4059795674516044800, 6436867623955512064, 2115337192179377792,
            4519819661567533696, 6644467032871428992, 1834595731470345472, 6469722508861870080, 2672030065446134656, 2001168543319218048, 
            2440660623886405504, 4682464743003293312, 4656677385699742208, 2957031626919939456, 5645504747023158400, 705098703608575744,
            588191888537402112, 5203822684102798592, 5419965878188457984, 6096705840454620800, 6268529198286308224, 6496325574947304448,
            6179115508262195200, 6041127310076589056, 4379227476242700928, 4121864828231575168, 4526229058440076288, 4316237348443952128,
            1823773960079216896, 1872588462410154240, 544927450310303104, 588191888537402112, 1226507282368609152]

In [29]:
len(koljonen_sourceids)

41

In [42]:
# write a gaia query to retrieve an astropy table of desired info for each source
query1_base = """
SELECT {columns}
FROM gaiadr3.gaia_source as gaia
WHERE (gaia.source_id = {source_id_cond})
"""

columns = 'source_id, ra, ra_error, dec, dec_error, parallax, pmra, pmdec'

koljonen_sourceids_str = ' OR gaia.source_id = '.join(str(x) for x in koljonen_sourceids)

query1 = query1_base.format(columns=columns, source_id_cond=koljonen_sourceids_str)
print(query1)


SELECT source_id, ra, ra_error, dec, dec_error, parallax, pmra, pmdec
FROM gaiadr3.gaia_source as gaia
WHERE (gaia.source_id = 455282205716288384 OR gaia.source_id = 3831382647922429952 OR gaia.source_id = 5367876720979404288 OR gaia.source_id = 3990037124929068032 OR gaia.source_id = 6128369984328414336 OR gaia.source_id = 6140785016794586752 OR gaia.source_id = 6098156298150016768 OR gaia.source_id = 4358428942492430336 OR gaia.source_id = 6025344817107454464 OR gaia.source_id = 4059795674516044800 OR gaia.source_id = 6436867623955512064 OR gaia.source_id = 2115337192179377792 OR gaia.source_id = 4519819661567533696 OR gaia.source_id = 6644467032871428992 OR gaia.source_id = 1834595731470345472 OR gaia.source_id = 6469722508861870080 OR gaia.source_id = 2672030065446134656 OR gaia.source_id = 2001168543319218048 OR gaia.source_id = 2440660623886405504 OR gaia.source_id = 4682464743003293312 OR gaia.source_id = 4656677385699742208 OR gaia.source_id = 2957031626919939456 OR gaia.sourc

In [43]:
job1 = Gaia.launch_job(query1)
results1 = job1.get_results()
results1.show_in_notebook()

idx,source_id,ra,ra_error,dec,dec_error,parallax,pmra,pmdec
,,deg,mas,deg,mas,mas,mas / yr,mas / yr
0,588191888537402112,143.83632970098836,1.4960221,9.009971735985376,1.1119037,0.7144949599085562,-7.6158085377880145,-2.9029869843872405
1,3831382647922429952,155.94870458335123,0.05794982,0.6446470547528212,0.05493738,0.6861708101572905,4.609895324338583,-17.281024260717615
2,455282205716288384,33.043636404275425,0.016298804,53.36078082215321,0.016981848,0.8584448143420856,-2.626652537633925,2.044329645061627
3,2957031626919939456,80.82054713445723,0.028570924,-25.460313037448238,0.031138053,0.4458118377520122,2.881414554645721,-4.583959680288839
4,1834595731470345472,299.39420990475,0.3567296,25.267236202653383,0.6594322,2.1497858656760136,-4.490694383250673,-12.290862414242437
5,6096705840454620800,214.37735925120347,0.04002687,-44.04932767896711,0.03322594,0.19737278858599186,-4.763750208392074,-5.103216631918363
6,2115337192179377792,274.1497258625874,0.09356995,45.17606864561869,0.09556594,0.22249783718568827,-0.05861630575280602,-4.399506732960086
7,3990037124929068032,162.18089987798868,0.30703384,23.664831000638443,0.38026184,0.48692630852515645,-15.445225592727764,-11.614618303913119
8,5645504747023158400,129.7100755896266,0.21409523,-28.465825811619446,0.26999032,0.4294249878990407,1.5039999803536581,-12.355483997749761


In [32]:
for i in range(len(koljonen_sourceids)):
    for j in range(len(results1['source_id'])):
        if koljonen_sourceids[i] == results1['source_id'][j]:
            if j == 0:
                print(i)

# interesting: koljonen lists gaia source 588191888537402112 twice, and matches it twice with psr J0935.3+0901 both times

24
39


In [33]:
# we next want to load in the ATNF pulsars and put it in a searchable astropy table
filename = '/home/billee/Binary-Pulsar-Distances/Binary_Pulsar_Distances/atnf_psrcat.csv'
atnf_table = QTable(names=('jname', 'ra (deg)', 'dec (deg)', 'pmra (mas/yr)', 'pmdec (mas/yr)', 'posepoch (jyear)'))
atnf_table['jname'].dtype = 'S2'
f = open(filename, "r")

line_number=0

# Loop through file of ATNF data and combine tables of Gaia matches into one supertable
for line in f:

    # the first two lines just describe the columns in the csv, so we want to skip them:
    if line_number > 1:
        # Parse input
        values = line.split(';')
        
        jname = values[1]
        ra = Angle(values[3], u.hourangle).deg
        dec = Angle(values[6], u.deg).deg
        if values[9] == '*':
            pmra = 0
        else: 
            pmra = u.Quantity(values[9], u.mas / u.yr)
        if values[12] == '*':
            pmdec = 0
        else: 
            pmdec = u.Quantity(values[12], u.mas / u.yr)
        if values[15] == '*':
            posepoch = 0
        else: 
            posepoch = Time(values[15], format='mjd').jyear

        atnf_table.add_row((jname, ra, dec, pmra, pmdec, posepoch))
    
    line_number += 1

In [34]:
atnf_table.show_in_notebook()

idx,jname,ra (deg),dec (deg),pmra (mas/yr),pmdec (mas/yr),posepoch (jyear)
0,J0002+6216,0.742375,62.26927777777777,0.0,0.0,2011.6673511293634
1,J0006+1834,1.52,18.583055555555557,0.0,0.0,1993.2511978097193
2,J0007+7303,1.757083333333333,73.05205555555555,0.0,0.0,2009.3292265571527
3,J0011+08,2.891666666666666,8.166666666666666,0.0,0.0,2014.996577686516
4,J0012+5431,3.097083333333333,54.52972222222222,0.0,0.0,2020.1711156741958
5,J0014+4746,3.5739583333333327,47.77594444444444,19.3,-19.7,1994.851471594798
6,J0021-0909,5.464458333333333,-9.166305555555557,0.0,0.0,2018.7364818617386
7,J0023+0923,5.820322908333333,9.389961222222222,-12.44,-6.16,2014.5119780971936
8,J0024-7204C,5.959810833333333,-72.075418,5.25,-2.55,2000.1519507186858
9,J0024-7204D,6.057837166666666,-72.07884788888889,5.25,-2.55,2000.1519507186858


In [36]:
# write the above code into a function: 
def gaia_crossmatch_atnf(matches, results1, atnf_table, mode='atnf_translate'):
    """Take a list of Gaia sources, and look for position matches in ATNF

    Args:
        matches (Table): Table to store matches in 
        results1 (Table): Table of gaia sources
        atnf_table (Table): Table of atnf sources
        mode (str, optional): Tell the function whether to translate the positions of the ATNF sources ('atnf_translate') or 
                              the Gaia sources ('gaia_translate')
    
    Returns:
        results (Table): results of the cross-match in an astropy table
    """


    gaia_epoch = Time('2016.0', format='jyear').jyear

    match_id = 0
    for gaia in results1:
        # number of pulsars matched to that gaia source
        number_of_matches = 0 

        # define SkyCoord object
        gaia_pos = SkyCoord(ra = gaia['ra'], dec = gaia['dec'], unit=(u.deg, u.deg), frame='icrs',
                            pm_ra_cosdec=gaia['pmra']*u.mas/u.yr, pm_dec=gaia['pmdec']*u.mas/u.yr)
        for psr in atnf_table:
            #---------------------------------update ATNF to Gaia pos---------------------------------
            # define atnf skycoord and epoch
            atnf_pos = SkyCoord(ra = psr['ra (deg)'], dec = psr['dec (deg)'], unit=(u.deg, u.deg), frame='icrs',
                                pm_ra_cosdec=psr['pmra (mas/yr)']*u.mas/u.yr, pm_dec=psr['pmdec (mas/yr)']*u.mas/u.yr)
            atnf_epoch = psr['posepoch (jyear)']

            if mode == 'gaia_translate':
                # calculate epoch difference
                year_diff = (atnf_epoch.tolist() * u.yr) - (gaia_epoch.tolist() * u.yr) # difference b/w epochs in years

                # pm * year_diff
                transra = gaia_pos.pm_ra_cosdec * year_diff
                transdec = gaia_pos.pm_dec * year_diff

                # update ra and dec
                ra_upd = (gaia_pos.ra.to(u.mas) + transra).to(u.deg)
                dec_upd = (gaia_pos.dec.to(u.mas) + transdec).to(u.deg)

                # update gaia skycoord obj
                gaia_pos_upd = SkyCoord(ra = ra_upd, dec = dec_upd, unit=(u.deg, u.deg), frame='icrs',
                                        pm_ra_cosdec=gaia['pmra']*u.mas/u.yr, pm_dec=gaia['pmdec']*u.mas/u.yr)
                
                # compare gaia position with psr position
                sep = atnf_pos.separation(gaia_pos_upd)
                sep_arcsec = sep.arcsecond
                if sep_arcsec <= 1.3:
                    matches.add_row((match_id, number_of_matches, sep_arcsec, psr['posepoch (jyear)'], gaia['source_id'], gaia_pos_upd.ra, gaia_pos_upd.dec,
                                    gaia_pos_upd.pm_ra_cosdec, gaia_pos_upd.pm_dec, gaia['parallax'], psr['jname'], atnf_pos.ra,
                                    atnf_pos.dec, atnf_pos.pm_ra_cosdec, atnf_pos.pm_dec))
                    match_id += 1
                    number_of_matches += 1
                
            if mode == 'atnf_translate':
                # calculate epoch difference
                year_diff = (gaia_epoch.tolist() * u.yr) - (atnf_epoch.tolist() * u.yr) # difference b/w epochs in years

                # pm * year_diff
                transra = atnf_pos.pm_ra_cosdec * year_diff
                transdec = atnf_pos.pm_dec * year_diff

                # update ra and dec
                ra_upd = (atnf_pos.ra.to(u.mas) + transra).to(u.deg)
                dec_upd = (atnf_pos.dec.to(u.mas) + transdec).to(u.deg)

                # update antf skycoord obj
                atnf_pos_upd = SkyCoord(ra = ra_upd, dec = dec_upd, unit=(u.deg, u.deg), frame='icrs',
                                        pm_ra_cosdec=psr['pmra (mas/yr)']*u.mas/u.yr, pm_dec=psr['pmdec (mas/yr)']*u.mas/u.yr)
                
                # compare gaia position with psr position
                sep = atnf_pos_upd.separation(gaia_pos)
                sep_arcsec = sep.arcsecond
                if sep_arcsec <= 1.3:
                    matches.add_row((match_id, number_of_matches, sep_arcsec, gaia_epoch, gaia['source_id'], gaia_pos.ra, gaia_pos.dec,
                                    gaia_pos.pm_ra_cosdec, gaia_pos.pm_dec, gaia['parallax'], psr['jname'], atnf_pos_upd.ra,
                                    atnf_pos_upd.dec, atnf_pos_upd.pm_ra_cosdec, atnf_pos_upd.pm_dec))
                    match_id += 1
                    number_of_matches += 1

        for row in matches:
            if row['gaia_source_id'] == gaia['source_id']:
                row['number_of_matches'] = number_of_matches
        

In [37]:
# match table to store all matches in
matches = Table(names=('match_id', 'number_of_matches', 'sep', 'epoch', 'gaia_source_id', 'gaia ra', 'gaia dec', 
                        'gaia pmra','gaia pmdec', 'gaia parallax', 'psr name', 'psr ra', 'psr dec', 'psr pmra', 'psr pmdec'),
                        dtype=(np.int64, np.int64, np.float64, np.float64, np.int64, np.float64, np.float64, np.float64, 
                                np.float64, np.float64, 'S2', np.float64, np.float64, np.float64, np.float64))


gaia_crossmatch_atnf(matches, results1, atnf_table, mode='atnf_translate')
matches.show_in_notebook()

idx,match_id,number_of_matches,sep,epoch,gaia_source_id,gaia ra,gaia dec,gaia pmra,gaia pmdec,gaia parallax,psr name,psr ra,psr dec,psr pmra,psr pmdec
0,0,1,0.0037746799443472367,2016.0,3831382647922429952,155.94870458335123,0.6446470547528214,4.6098953243385825,-17.281024260717615,0.6861708101572905,J1023+0038,155.94870531314928,0.6446478076545745,4.76,-17.34
1,1,1,0.02984912095102211,2016.0,1834595731470345472,299.39420990475,25.267236202653383,-4.490694383250673,-12.290862414242435,2.1497858656760136,J1957+2516,299.3942145833333,25.267243333333333,0.0,0.0
2,2,1,0.4007888533985052,2016.0,6096705840454620800,214.37735925120344,-44.04932767896711,-4.763750208392074,-5.103216631918363,0.19737278858599186,J1417-4402,214.3774986899764,-44.04927919929273,-4.7,-5.1
3,3,1,0.01576971703161092,2016.0,2115337192179377792,274.1497258625874,45.17606864561869,-0.05861630575280605,-4.399506732960086,0.22249783718568827,J1816+4510,274.1497319072172,45.17606966153321,5.3,-3.0
4,4,1,0.00502175825143581,2016.0,3990037124929068032,162.18089987798868,23.664831000638447,-15.445225592727764,-11.614618303913119,0.48692630852515645,J1048+2339,162.18089985966807,23.664829605806528,-18.999999999999996,-9.400000000000002
5,5,1,0.001206740366148455,2016.0,6469722508861870080,309.8957015427482,-56.2859099942261,3.861062488612514,-15.179687664578143,0.4886902269612317,J2039-5617,309.895701,-56.285909847222214,4.199999999999999,-14.899999999999999
6,6,1,0.17789598049762198,2016.0,4316237348443952128,292.18909202584297,12.764810213495082,-0.3500554761478733,-4.621352584743756,0.1483331667098763,J1928+1245,292.18913999999995,12.764826111111114,0.0,0.0
7,7,1,0.31376823714631175,2016.0,4059795674516044800,260.8465812119027,-28.632665334243985,-11.73376052135494,-24.049698930825514,1.0663413131826731,J1723-2837,260.84659133564145,-28.632578630530837,-11.71,-23.990000000000002
8,8,1,3.986747714792336e-05,2016.0,5367876720979404288,159.12589636347832,-43.885757007326994,-11.613278553276848,2.879299753464205,0.36030013136680694,J1036-4353,159.12589637499997,-43.88575699999999,0.0,0.0
9,9,1,0.10998823927692251,2016.0,4526229058440076288,272.6553648133798,17.743710793760194,7.537937766409877,-4.189163973548999,0.6476206297078896,J1810+1744,272.6553333333333,17.74371666666666,0.0,0.0


In [41]:
pm0 = matches[matches['psr pmdec']==0.0]
pm0.show_in_notebook()

idx,match_id,number_of_matches,sep,epoch,gaia_source_id,gaia ra,gaia dec,gaia pmra,gaia pmdec,gaia parallax,psr name,psr ra,psr dec,psr pmra,psr pmdec
0,1,1,0.02984912095102211,2016.0,1834595731470345472,299.39420990475,25.267236202653383,-4.490694383250673,-12.290862414242435,2.1497858656760136,J1957+2516,299.3942145833333,25.267243333333333,0.0,0.0
1,6,1,0.17789598049762198,2016.0,4316237348443952128,292.18909202584297,12.764810213495082,-0.3500554761478733,-4.621352584743756,0.1483331667098763,J1928+1245,292.18913999999995,12.764826111111114,0.0,0.0
2,8,1,3.986747714792336e-05,2016.0,5367876720979404288,159.12589636347832,-43.885757007326994,-11.613278553276848,2.879299753464205,0.36030013136680694,J1036-4353,159.12589637499997,-43.88575699999999,0.0,0.0
3,9,1,0.10998823927692251,2016.0,4526229058440076288,272.6553648133798,17.743710793760194,7.537937766409877,-4.189163973548999,0.6476206297078896,J1810+1744,272.6553333333333,17.74371666666666,0.0,0.0
4,14,1,6.115353578405448e-05,2016.0,6041127310076589056,238.91941063988847,-29.141228677596377,0.0,0.0,0.0,J1555-2908,238.91941062499995,-29.141228666666667,0.0,0.0
5,17,1,0.09765262552062232,2016.0,2001168543319218048,333.8861955818804,51.593454638828675,0.014390063442942025,2.240303327249447,0.30157491462634983,J2215+5135,333.88620166666664,51.593427777777784,0.0,0.0
6,18,1,0.6152377493433218,2016.0,4519819661567533696,287.2387172890098,21.083919807695853,2.695292539001098,-7.90357473795299,-2.168909640804543,J1908+2105,287.23872499999993,21.084090555555562,0.0,0.0
7,20,1,0.33254063280130913,2016.0,6140785016794586752,196.73446719720647,-40.589833028920324,-6.185437320518144,4.1610439339172265,0.31328711014545335,J1306-4035,196.73458333333323,-40.58980555555556,0.0,0.0
8,21,1,0.04433780323640841,2016.0,6179115508262195200,197.94050550830735,-34.50843790660204,-6.133366846443238,-5.144624360003818,1.931193654662938,J1311-3430,197.94051749999997,-34.508430555555556,0.0,0.0
